In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import tushare as ts
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_2Dsyn1.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
#type(data.shape[1])

In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,-0.214242,-0.071069,0.044274,0.061571,-0.127298,-0.106092,-0.048952,-0.076331,-0.229400,0.106868,...,0.008154,-0.371059,0.244589,0.017343,0.040964,0.008304,0.246045,-0.152729,-0.247024,0.012803
1,-0.025840,0.018123,-0.262684,0.064054,0.050128,0.077492,0.137940,0.147687,0.300642,-0.073297,...,-0.062408,-0.159959,0.298935,0.149444,0.144854,-0.137243,-0.113464,0.040892,0.048967,0.108723
2,-0.059878,-0.097412,-0.117858,-0.008519,0.032316,-0.115909,-0.019436,0.456654,0.167180,-0.161088,...,0.362173,0.267039,-0.167836,-0.087735,0.049714,-0.000850,-0.039238,-0.046553,-0.056353,0.202016
3,0.179650,-0.186147,0.062173,-0.076138,0.233298,-0.003869,0.068773,0.083363,-0.038388,-0.153099,...,-0.236150,-0.181563,-0.053868,-0.042065,0.101236,0.292420,0.012626,-0.174028,0.120366,0.081407
4,0.431270,0.056588,0.009886,0.325598,-0.026840,0.710478,0.122607,0.057843,0.221965,-0.113067,...,0.212242,0.234083,0.089941,-0.224374,0.045691,0.119132,-0.075106,0.011380,0.044389,0.265852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33137,0.012570,0.071684,0.134780,0.058359,0.080002,-0.336990,0.035481,-0.322269,0.095744,0.037659,...,-0.030904,-0.124197,-0.137501,-0.062988,0.163377,-0.122231,-0.162339,0.396063,0.113521,-0.261529
33138,-0.173373,-0.129791,-0.087971,0.112266,-0.073055,0.256870,-0.008347,-0.230685,-0.110658,0.011698,...,-0.197360,0.119050,0.244596,-0.170639,0.187981,0.265022,0.136199,0.014366,-0.087579,-0.005440
33139,-0.058899,0.015288,-0.055067,0.032984,-0.006103,0.076559,-0.244653,-0.257687,-0.163633,0.102046,...,-0.109157,0.360627,0.068984,-0.048404,-0.171367,0.011185,-0.062593,0.109995,0.107560,0.125816
33140,-0.173949,0.045912,-0.126463,-0.070584,-0.061190,-0.085109,0.047336,-0.020572,-0.073108,-0.013986,...,-0.266267,-0.178294,-0.159218,0.012947,-0.245111,0.031110,-0.026095,-0.053649,-0.197968,0.226570


In [7]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [8]:
data.shape

torch.Size([33142, 2304])

In [9]:
#将前80%作为训练集，后20%作为测试集
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 48
w2 = 48

train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([26513, 2304])
torch.Size([6629, 2304])


In [10]:
#定义Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [11]:
class PAB(nn.Module):  ##position attention block
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fcb1 = FCB(input_size, output_size, dropout)
        self.fcb2 = FCB(input_size, output_size, dropout)
        self.fcb3 = FCB(input_size, output_size, dropout)
        self.fcb4 = FCB(input_size, output_size, dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x1 = self.fcb1(x)
        x2 = self.fcb2(x)
        x3 = self.fcb3(x)
        x4 = self.fcb4(x)
        

        x = x1*x2
        x = self.softmax(x)
        x = x*x3
        x = x+x4
        
        return x   

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.1):
        super().__init__()

        self.cov1d = nn.Conv1d(in_channels=1, out_channels= 64 ,kernel_size = 5, padding=0, stride=1)

        self.pab1 = PAB(int(input_size-5+1), 128, dropout)
        self.fcb1 = FCB(128, 128, dropout)

        self.pab2 = PAB(128, 64, dropout)
        self.fcb2 = FCB(64, 64, dropout)

        self.pab3 = PAB(64, 32, dropout)
        self.fcb3 = FCB(32, 32, dropout)

        self.pab4 = PAB(32, 16, dropout)
        self.fcb4 = FCB(16, 16, dropout)

        self.pab5 = PAB(16, 8, dropout)
        self.fcb5 = FCB(8, 8, dropout)

        self.pab6 = PAB(8, 4, dropout)
        self.fcb6 = FCB(4, 4, dropout)
        
        self.fcb7 = FCB(4, 4, dropout)
        self.pab7 = PAB(4, 4, dropout)
        

        


    def forward(self, x):

        x = x.reshape(x.shape[0],1,x.shape[1])
        x = self.cov1d(x)
        x = x.permute(0,2,1)
        x = nn.AdaptiveAvgPool1d(1)(x)
        x = x.squeeze()

        x1 = self.pab1(x)
        x13 = self.fcb1(x1)

        x2 = self.pab2(x1)
        x12 = self.fcb2(x2)

        x3 = self.pab3(x2)
        x11 = self.fcb3(x3)

        x4 = self.pab4(x3)
        x10 = self.fcb4(x4)

        x5 = self.pab5(x4)
        x9 = self.fcb5(x5)

        x6 = self.pab6(x5)
        x8 = self.fcb6(x6)

        x7 = self.fcb7(x6)
        x7 = self.pab7(x7)
        

        

        return x7,x8,x9,x10,x11,x12,x13

In [13]:
class CustomActivation(torch.autograd.Function):    
    @staticmethod
    def forward(ctx, x):
        # 计算正向传播结果
        ctx.save_for_backward(x)
        y = 11/3 * (x - torch.tanh(x)) * torch.cos(x/2)
        return y
    
    @staticmethod
    def backward(ctx, grad_output):
        # 计算反向传播结果
        x, = ctx.saved_tensors
        grad_x = grad_output * (11/3 * ((torch.cos(x/2)*(1 - torch.pow(torch.cosh(x), -2)))+ (x - torch.tanh(x))*(-0.5*torch.sin(x/2))))
                                       
        return grad_x
    
class CustomActivationModule(torch.nn.Module):
    def forward(self, x):
        return CustomActivation.apply(x)  
    

activation = CustomActivationModule()    

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.1):
        super().__init__()

        self.pab1 = PAB(8, 8, dropout)
        self.pab2 = PAB(16, 16, dropout)
        self.pab3 = PAB(32, 32, dropout)
        self.pab4 = PAB(64, 64, dropout)
        self.pab5 = PAB(128, 128, dropout)
        #self.pab5 = PAB(256, 256, dropout)

        self.fc = nn.Linear(256,output_size)
        #self.activation = nn.Identity()
        self.activation = activation


    def forward(self, x7,x8,x9,x10,x11,x12,x13):
        
        x = torch.cat((x7,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x9),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x10),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x11),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x12),dim=1)
        x = self.pab5(x)
        x = torch.cat((x,x13),dim=1)
        x = self.fc(x)
        #x.requires_grad_(True)
        x = self.activation(x)
        #x.sum().backward()
        return x

In [15]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x7,x8,x9,x10,x11,x12,x13= self.encoder(x)
        x = self.decoder(x7,x8,x9,x10,x11,x12,x13)

        return x

In [16]:
class Loss(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        temp = output*N
        temp1 = data.shape[1]*(temp.sum(1))
        temp2 = (output.sum(1))*(N.sum(1))
        temp3 = torch.pow(data.shape[1]*((output**2).sum(1)) - ((output.sum(1))**2),1/2)
        temp4 = torch.pow(data.shape[1]*((N**2).sum(1))-((N.sum(1))**2),1/2) 
        loss = (temp1-temp2)/(temp3*temp4)

        a = torch.min(loss)
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))
        
        return self.r*a+(1-self.r)*b

In [17]:
class Loss0(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))
        K = output.shape[0]
        y2_mean = torch.mean(output)
        n2_mean = torch.mean(N)
        covariance = 0.0
        variance_y2 = 0.0
        variance_n2 = 0.0
        for j in range(K):
            y2_sample = output[j, :]
            n2_sample = N[j, :]
            covariance += ((y2_sample - y2_mean) * (n2_sample - n2_mean))
            variance_y2 += ((y2_sample - y2_mean) ** 2)
            variance_n2 += ((n2_sample - n2_mean) ** 2)
            lcc_value = (covariance / torch.sqrt(variance_y2 * variance_n2)) ** 2
            lcc_value = torch.mean(lcc_value)
        return self.r*lcc_value+(1-self.r)*b

In [18]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2,w1*w2).to(device)
#将模型转移到GPU上
#criterion = MeanHuberLoss(delta=1.3)
#criterion = WelschLoss(delta=0.5)
#criterion = Loss0(delta=0.46,r=0.05)#0.5 and 0.2,SNR:-8dB ok
criterion = Loss0(delta=0.46,r=0.01)
#criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [19]:
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 # 总训练轮数
#num_batch_train = 0
for epoch in range(num_epochs):
  #train_bar = tqdm(train_loader)
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    # 数据转到device
    train_batch = batch[0].to(device)
    
    # 训练步骤  
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()
    
    train_loss += loss.item()
    #num_batch_train +=1
  #train_loss除以所有bacth个数
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  # 验证
  valid_loss = 0.0
  #num_batch_vaild = 0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    #for batch in vaild_loader:
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
      #num_batch_vaild += 1
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        #es_cnt = 0
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  

    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.0992, Valid Loss: 0.0965
loss_train:  [0.09920534301235015]
vaild_train:  [0.09653103179656543]
Epoch [2/100], Train Loss: 0.0891, Valid Loss: 0.0914
loss_train:  [0.09920534301235015, 0.08908541243837541]
vaild_train:  [0.09653103179656543, 0.09138313358506331]
Epoch [3/100], Train Loss: 0.0826, Valid Loss: 0.0857
loss_train:  [0.09920534301235015, 0.08908541243837541, 0.08258074100118086]
vaild_train:  [0.09653103179656543, 0.09138313358506331, 0.08571263617621018]
Epoch [4/100], Train Loss: 0.0795, Valid Loss: 0.0833
loss_train:  [0.09920534301235015, 0.08908541243837541, 0.08258074100118086, 0.07952102110328445]
vaild_train:  [0.09653103179656543, 0.09138313358506331, 0.08571263617621018, 0.08330543138659917]
Epoch [5/100], Train Loss: 0.0773, Valid Loss: 0.0816
loss_train:  [0.09920534301235015, 0.08908541243837541, 0.08258074100118086, 0.07952102110328445, 0.07728398374046187]
vaild_train:  [0.09653103179656543, 0.09138313358506331, 0.0857126361762101

In [20]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)

loss = pd.concat([loss_train,loss_vaild],axis=1)

In [21]:
loss.columns = ['train_loss','vaild_loss']

In [22]:
torch.save(model.state_dict(), r'.\model_2dsyn1.pth')

In [23]:
model = AutoEncoder(w1*w2,w1*w2).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_2dsyn1.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    #loss = criterion(output, data)
print(output.shape)

torch.Size([33142, 2304])


In [24]:
output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)

In [25]:
loss.to_csv(r'loss_2dsyn1.csv',index=False)

In [26]:
output.to_csv(r'./output_2dsyn1.csv',index=None,header=None)

In [27]:
#loss_0 = []
#start_time = time.time()
#es_cnt = 0
#es_thres = 5
#prev_train_loss = float('inf')
#loss_train = []
#loss_vaild = []
#num_epochs = 100 # 总训练轮数
##num_batch_train = 0
#for epoch in range(num_epochs):
#  #train_bar = tqdm(train_loader)
#  train_loss = 0.0
#  
#  for i , (batch) in enumerate(train_loader):
#
#    # 数据转到device
#    train_batch = batch[0].to(device)
#    
#    # 训练步骤  
#    optimizer.zero_grad()
#    outputs = model(train_batch)
#    loss = criterion(outputs, train_batch)
#    loss.backward() 
#    optimizer.step()
#
#    loss_0.append(loss.item())
#
#    train_loss += loss.item()
#    #num_batch_train +=1
#  #train_loss除以所有bacth个数
#  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
#  loss_train.append(train_loss)
#    
#
#
#
#  # 验证
#  valid_loss = 0.0
#  #num_batch_vaild = 0
#  with torch.no_grad():
#    for i , (batch) in enumerate(vaild_loader):
#    #for batch in vaild_loader:
#    
#      val_batch = batch[0].to(device)
#      
#      outputs = model(val_batch)
#      loss = criterion(outputs, val_batch)
#      valid_loss += loss.item()
#      #num_batch_vaild += 1
#    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
#    loss_vaild.append(valid_loss)
#    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))
#
#    
#    # Early stopping
#    if train_loss - prev_train_loss >= 0:
#        es_cnt += 1
#    else:
#        #es_cnt = 0
#        pass
#
#    if es_cnt >= es_thres:
#        print(f"Early stopped at epoch {epoch}, train loss stop improving")
#        break  
#    
#
#
##   if epoch == 10:
##     torch.save(model.state_dict(), r'.\model_epochs10.pth')
##   elif epoch == 20:
##     torch.save(model.state_dict(), r'.\model_epochs20.pth')
##   elif epoch == 30:
##     torch.save(model.state_dict(), r'.\model_epochs30.pth')
##   elif epoch == 40:
##     torch.save(model.state_dict(), r'.\model_epochs40.pth')
##   elif epoch == 50:
##     torch.save(model.state_dict(), r'.\model_epochs50.pth')
##   elif epoch == 60:
##     torch.save(model.state_dict(), r'.\model_epochs60.pth')
##   elif epoch == 70:
##     torch.save(model.state_dict(), r'.\model_epochs70.pth')
##   elif epoch == 80:
##     torch.save(model.state_dict(), r'.\model_epochs80.pth')
##   elif epoch == 90:
##     torch.save(model.state_dict(), r'.\model_epochs90.pth')
##   elif epoch == 100:
##     torch.save(model.state_dict(), r'.\model_epochs100.pth')
##   else:
##       pass
#
#    prev_train_loss = train_loss
#  print('loss_train: ', loss_train)
#  print('vaild_train: ',loss_vaild)          
#print("Training finished")
#current_time = time.time()
#time_sum = current_time-start_time
#print(time_sum)